In [98]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import xgboost as xgb

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [99]:
df = pd.read_excel("../data/raw/entrenamiento.xlsx")
df = df.rename(columns={"C7.1": "C8", "des": "QUALITY"})
features = df.drop(columns=["QUALITY"])
target = df["QUALITY"]

In [100]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
C1,3684.0,6.857166,0.846430,3.800000,6.300000,6.800000,7.300000,14.200000
C2,3664.0,0.421440,0.988207,0.076961,0.190620,0.231112,0.285179,6.913737
C3,3648.0,0.284054,0.086887,0.000000,0.239017,0.277632,0.329304,0.802002
C4,3636.0,6.424642,5.084063,0.600000,1.700000,5.300000,9.900000,65.800000
C5,3659.0,40.685294,25.106022,0.020000,33.000000,41.000000,49.000000,346.000000
C6,3655.0,35.216142,17.167346,2.000000,23.000000,34.000000,46.000000,289.000000
C7,3645.0,4.885591,0.341723,2.302585,4.691348,4.912655,5.129899,6.089045
C8,3646.0,65.824651,245.359289,0.987110,0.991800,0.994100,0.996825,1001.000000
C9,3658.0,3.188291,0.152155,2.720000,3.090000,3.180000,3.280000,3.820000
C10,3639.0,0.395421,0.074535,0.198851,0.343590,0.385262,0.438255,0.732368


In [101]:
# Calculando la cantidad de valores atípicos según los criterios dados para cada característica

# C2
outliers_C2 = df[df['C2'] > 0.29]

# C5
outliers_C5_below = df[df['C5'] < 33]
outliers_C5_above = df[df['C5'] > 49]

# C4
outliers_C4 = df[df['C4'] > 9.9]

# C6
outliers_C6_below = df[df['C6'] < 23]
outliers_C6_above = df[df['C6'] > 46]

# C8
outliers_C8 = df[df['C8'] > 1]

outliers_counts = {
    'C2': len(outliers_C2),
    'C5_below': len(outliers_C5_below),
    'C5_above': len(outliers_C5_above),
    'C4': len(outliers_C4),
    'C6_below': len(outliers_C6_below),
    'C6_above': len(outliers_C6_above),
    'C8': len(outliers_C8)
}

outliers_counts

{'C2': 853,
 'C5_below': 906,
 'C5_above': 820,
 'C4': 906,
 'C6_below': 873,
 'C6_above': 879,
 'C8': 290}

In [102]:
# Filtrando los datos atípicos para cada característica y extrayendo la columna "QUALITY"
quality_c2 = df[df['C2'] > 0.29]['QUALITY']
quality_c5_below = df[df['C5'] < 33]['QUALITY']
quality_c5_above = df[df['C5'] > 49]['QUALITY']
quality_c4 = df[df['C4'] > 9.9]['QUALITY']
quality_c6_below = df[df['C6'] < 23]['QUALITY']
quality_c6_above = df[df['C6'] > 46]['QUALITY']
quality_c8 = df[df['C8'] > 1]['QUALITY']

# Creando un diccionario con las distribuciones de "QUALITY" para cada conjunto de observaciones atípicas
quality_distributions = {
    'C2 > 0.29': quality_c2,
    'C5 < 33': quality_c5_below,
    'C5 > 49': quality_c5_above,
    'C4 > 9.9': quality_c4,
    'C6 < 23': quality_c6_below,
    'C6 > 46': quality_c6_above,
    'C8 > 1': quality_c8
}

# Retornando las descripciones de estas distribuciones
quality_descriptions = {key: value.describe() for key, value in quality_distributions.items()}
quality_descriptions

{'C2 > 0.29': count    853.000000
 mean       5.728019
 std        0.980383
 min        3.000000
 25%        5.000000
 50%        6.000000
 75%        6.000000
 max        9.000000
 Name: QUALITY, dtype: float64,
 'C5 < 33': count    906.000000
 mean       6.073951
 std        0.923628
 min        3.000000
 25%        5.000000
 50%        6.000000
 75%        7.000000
 max        9.000000
 Name: QUALITY, dtype: float64,
 'C5 > 49': count    820.000000
 mean       5.580488
 std        0.817758
 min        3.000000
 25%        5.000000
 50%        6.000000
 75%        6.000000
 max        8.000000
 Name: QUALITY, dtype: float64,
 'C4 > 9.9': count    906.000000
 mean       5.730684
 std        0.817142
 min        3.000000
 25%        5.000000
 50%        6.000000
 75%        6.000000
 max        9.000000
 Name: QUALITY, dtype: float64,
 'C6 < 23': count    873.000000
 mean       5.681558
 std        0.941812
 min        3.000000
 25%        5.000000
 50%        6.000000
 75%        6.00

In [103]:
# Corregir los valores de C8 mayores a 1 dividiéndolos por 1000 en el DataFrame original
df["C8"] = df["C8"].apply(lambda x: x / 1000 if x > 1 else x)

# Verificar el cambio
df["C8"].describe()

count    3646.000000
mean        0.979775
std         0.117781
min         0.001000
25%         0.991660
50%         0.993630
75%         0.995900
max         1.001000
Name: C8, dtype: float64

In [104]:
# Imputar datos faltantes con la mediana
df_filled = df.copy()
for column in df_filled.columns:
    df_filled[column].fillna(df_filled[column].median(), inplace=True)

# Verificar si quedan datos faltantes
missing_values = df_filled.isnull().sum()

missing_values

C1         0
C2         0
C3         0
C4         0
C5         0
C6         0
C7         0
C8         0
C9         0
C10        0
C11        0
QUALITY    0
dtype: int64

In [105]:
df_filled

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,QUALITY
0,6.8,0.190620,0.307485,18.1,46.00,32.0,4.897840,1.00000,3.27,0.392042,8.8,5
1,6.2,0.246860,0.277632,5.0,43.00,50.0,5.241747,0.99318,3.23,0.494696,10.8,6
2,6.7,0.350657,0.329304,12.1,0.04,61.0,5.517453,0.99794,3.31,0.457425,9.7,5
3,6.8,0.254642,0.405465,13.3,53.00,48.0,5.273000,0.99740,3.09,0.371564,9.4,5
4,6.6,0.215111,0.239017,15.8,35.00,46.0,5.241747,0.99820,3.24,0.412110,9.2,5
...,...,...,...,...,...,...,...,...,...,...,...,...
3913,7.7,0.231112,0.215111,11.2,31.00,41.0,5.123964,0.99480,3.12,0.357674,11.3,7
3914,8.1,0.378436,0.270027,1.7,52.00,50.0,5.214936,0.99230,3.03,0.350657,11.2,5
3915,6.1,0.148420,0.254642,6.0,0.03,29.0,4.976734,0.99474,3.68,0.378436,10.7,6
3916,8.2,0.239017,0.329304,7.8,41.00,49.0,5.342334,0.99760,3.31,0.412110,9.5,6


In [106]:
X = df_filled.drop(columns=['QUALITY'])
y = df_filled['QUALITY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [134]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

lr = LinearRegression()
lr.fit(X_train_std, y_train)
predictions_lr = lr.predict(X_test_std)
mae_lr = mean_absolute_error(y_test, predictions_lr)
print(f"MAE for Linear Regression: {mae_lr:.4f}")

MAE for Linear Regression: 0.6218


In [108]:
from sklearn.linear_model import Ridge

ridge = Ridge(random_state=42)
ridge.fit(X_train_std, y_train)
predictions_ridge = ridge.predict(X_test_std)
mae_ridge = mean_absolute_error(y_test, predictions_ridge)
print(f"MAE for Ridge Regression: {mae_ridge:.4f}")

MAE for Ridge Regression: 0.6218


In [109]:
from sklearn.linear_model import Lasso

lasso = Lasso(random_state=42)
lasso.fit(X_train_std, y_train)
predictions_lasso = lasso.predict(X_test_std)
mae_lasso = mean_absolute_error(y_test, predictions_lasso)
print(f"MAE for Lasso Regression: {mae_lasso:.4f}")

MAE for Lasso Regression: 0.6500


In [110]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor(random_state=42)
dt.fit(X_train_std, y_train)
predictions_dt = dt.predict(X_test_std)
mae_dt = mean_absolute_error(y_test, predictions_dt)
print(f"MAE for Decision Tree: {mae_dt:.4f}")

MAE for Decision Tree: 0.6224


## Random Forest

In [111]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=42)
rf.fit(X_train_std, y_train)
predictions_rf = rf.predict(X_test_std)
mae_rf = mean_absolute_error(y_test, predictions_rf)
print(f"MAE for Random Forest: {mae_rf:.4f}")

MAE for Random Forest: 0.5089


In [112]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Definir los hiperparámetros y sus posibles valores
param_grid_forest = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 6, 7, 8, 9, 10],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [1, 2, 3, 4],
    'bootstrap': [True, False]
}

# Instanciar el modelo y el GridSearchCV
forest = RandomForestRegressor(random_state=42)
grid_search_forest = GridSearchCV(forest, param_grid_forest, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

# Ajustar el GridSearchCV al conjunto de datos estandarizado
grid_search_forest.fit(X_train_std, y_train)

# Obtener los mejores hiperparámetros y el mejor MAE
best_params_forest = grid_search_forest.best_params_
best_mae_forest = -grid_search_forest.best_score_

best_params_forest, best_mae_forest

({'bootstrap': True,
  'max_depth': None,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'n_estimators': 150},
 0.5200567130868123)

In [113]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150, random_state=42)
rf.fit(X_train_std, y_train)
predictions_rf = rf.predict(X_train_std)
mae_rf = mean_absolute_error(y_train, predictions_rf)
print(f"MAE for Random Forest: {mae_rf:.4f}")

MAE for Random Forest: 0.1862


In [114]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    max_depth=30,
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=100,
    bootstrap=True,
    random_state=42)
rf.fit(X_train_std, y_train)
predictions_rf = rf.predict(X_train_std)
mae_rf = mean_absolute_error(y_train, predictions_rf)
print(f"MAE for Random Forest: {mae_rf:.4f}")

MAE for Random Forest: 0.1871


## SVR

In [115]:
from sklearn.svm import SVR

svr = SVR()
svr.fit(X_train_std, y_train)
predictions_svr = svr.predict(X_test_std)
mae_svr = mean_absolute_error(y_test, predictions_svr)
print(f"MAE for SVR: {mae_svr:.4f}")

MAE for SVR: 0.5674


## AdaBoost

In [116]:
from sklearn.ensemble import AdaBoostRegressor

adaboost = AdaBoostRegressor(random_state=42)
adaboost.fit(X_train_std, y_train)
predictions_adaboost = adaboost.predict(X_test_std)
mae_adaboost = mean_absolute_error(y_test, predictions_adaboost)
print(f"MAE for AdaBoost: {mae_adaboost:.4f}")

MAE for AdaBoost: 0.5971


## Gradient Boosting

In [117]:
from sklearn.ensemble import GradientBoostingRegressor

gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train_std, y_train)
predictions_gb = gb.predict(X_test_std)
mae_gb = mean_absolute_error(y_test, predictions_gb)
print(f"MAE for Gradient Boosting: {mae_gb:.4f}")

MAE for Gradient Boosting: 0.5603


In [118]:
# Hiperparámetros optimizados
gb_model_std = GradientBoostingRegressor(
    learning_rate=0.1,
    max_depth=7,
    max_features='sqrt',
    min_samples_leaf=3,
    min_samples_split=2,
    n_estimators=150,
    subsample=1.0,
    random_state=42)  # Puedes ajustar los hiperparámetros según sea necesario
gb_model_std.fit(X_train_std, y_train)

# Hacer predicciones en el conjunto de entrenamiento
y_train_pred_gb = gb_model_std.predict(X_train_std)

# Calcular el MAE para las predicciones
mae_gb_std = mean_absolute_error(y_train, y_train_pred_gb)
print(f'MAE of Gradient Boosting Regression: {mae_gb_std:.4}')

MAE of Gradient Boosting Regression: 0.2112


In [119]:
# Hiperparámetros optimizados
gb_model_std = GradientBoostingRegressor(
    learning_rate=0.1,
    max_depth=7,
    max_features='sqrt',
    min_samples_leaf=3,
    min_samples_split=2,
    n_estimators=100,
    subsample=0.9,
    random_state=42)  # Puedes ajustar los hiperparámetros según sea necesario
gb_model_std.fit(X_train_std, y_train)

# Hacer predicciones en el conjunto de entrenamiento
y_train_pred_gb = gb_model_std.predict(X_train_std)

# Calcular el MAE para las predicciones
mae_gb_std = mean_absolute_error(y_train, y_train_pred_gb)
print(f'MAE of Gradient Boosting Regression: {mae_gb_std:.4}')

MAE of Gradient Boosting Regression: 0.2622


## XGBoost

In [120]:
# Hiperparámetros optimizados
# Aplicar los hiperparámetros óptimos al modelo XGBoost
optimized_xgb = xgb.XGBRegressor(objective='reg:squarederror',
                                 subsample=1.0,
                                 min_child_weight=4,
                                 max_depth=9,
                                 learning_rate=0.1,
                                 gamma=0,
                                 colsample_bytree=0.6,
                                 random_state=42)

# Entrenar el modelo con los datos estandarizados
optimized_xgb.fit(X_train_std, y_train)

# Hacer predicciones en el conjunto de entrenamiento
xgb_optimized_predictions = optimized_xgb.predict(X_train_std)

# Calcular el MAE
mae_optimized_xgb = mean_absolute_error(y_train, xgb_optimized_predictions)
mae_optimized_xgb
print(f'MAE of XGBoost: {mae_optimized_xgb:.4}')

MAE of XGBoost: 0.163


In [121]:
# Hiperparámetros optimizados
# Aplicar los hiperparámetros óptimos al modelo XGBoost
optimized_xgb = xgb.XGBRegressor(objective='reg:squarederror',
                                 subsample=1.0,
                                 min_child_weight=4,
                                 max_depth=9,
                                 learning_rate=0.1,
                                 gamma=0,
                                 colsample_bytree=0.6,
                                 random_state=42)

# Entrenar el modelo con los datos estandarizados
optimized_xgb.fit(X_test_std, y_test)

# Hacer predicciones en el conjunto de entrenamiento
xgb_optimized_predictions = optimized_xgb.predict(X_test_std)

# Calcular el MAE
mae_optimized_xgb = mean_absolute_error(y_test, xgb_optimized_predictions)
mae_optimized_xgb
print(f'MAE of XGBoost: {mae_optimized_xgb:.4}')

MAE of XGBoost: 0.06777


In [122]:
# Hiperparámetros optimizados
# Aplicar los hiperparámetros óptimos al modelo XGBoost
optimized_xgb = xgb.XGBRegressor(objective='reg:squarederror',
                                 subsample=0.8,
                                 min_child_weight=4,
                                 max_depth=9,
                                 learning_rate=0.1,
                                 gamma=0,
                                 colsample_bytree=0.8,
                                 random_state=42)

# Entrenar el modelo con los datos estandarizados
optimized_xgb.fit(X_train_std, y_train)

# Hacer predicciones en el conjunto de entrenamiento
xgb_optimized_predictions = optimized_xgb.predict(X_train_std)

# Calcular el MAE
mae_optimized_xgb = mean_absolute_error(y_train, xgb_optimized_predictions)
mae_optimized_xgb
print(f'MAE of XGBoost: {mae_optimized_xgb:.4}')

MAE of XGBoost: 0.1516


## Stacking

In [123]:
from sklearn.ensemble import StackingRegressor

base_learners = [
    ('rf', RandomForestRegressor(random_state=42)),
    ('gb', GradientBoostingRegressor(random_state=42)),
    ('ridge', Ridge(random_state=42))
]

stacking_regressor = StackingRegressor(estimators=base_learners, final_estimator=LinearRegression())
stacking_regressor.fit(X_train_std, y_train)
predictions_stack = stacking_regressor.predict(X_test_std)
mae_stack = mean_absolute_error(y_test, predictions_stack)
print(f"MAE for Stacking Regressor: {mae_stack:.4f}")

MAE for Stacking Regressor: 0.5105


In [124]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Define the base models
base_models = [
    ("xgb", xgb.XGBRegressor(objective='reg:squarederror', random_state=42)),
    ("rf",
     RandomForestRegressor(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150, random_state=42)),
    ("gb", GradientBoostingRegressor(learning_rate=0.1, max_depth=7, max_features='sqrt', min_samples_leaf=3,
                                     min_samples_split=2, n_estimators=150, subsample=1.0, random_state=42))
]

# Initialize the Stacking Regressor with the base models and a linear meta-model
stacked_model = StackingRegressor(estimators=base_models, final_estimator=LinearRegression())

# Train the model
stacked_model.fit(X_train_std, y_train)

# Predict on the training set
stacked_train_predictions = stacked_model.predict(X_train_std)

# Calculate the MAE for the Stacking Regressor
mae_stacked = mean_absolute_error(y_train, stacked_train_predictions)
print(f'MAE of Stacking Regressor: {mae_stacked:.6f}')

MAE of Stacking Regressor: 0.173959


In [125]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Define the base models
base_models = [
    ("xgb", xgb.XGBRegressor(objective='reg:squarederror', random_state=42)),
    ("rf",
     RandomForestRegressor(max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150, random_state=42)),
    ("gb", GradientBoostingRegressor(learning_rate=0.1, max_depth=7, max_features='sqrt', min_samples_leaf=3,
                                     min_samples_split=2, n_estimators=150, subsample=1.0, random_state=42))
]

# Initialize the Stacking Regressor with the base models and a linear meta-model
stacked_model = StackingRegressor(estimators=base_models, final_estimator=LinearRegression())

# Train the model
stacked_model.fit(X_train_std, y_train)

# Predict on the training set
stacked_train_predictions = stacked_model.predict(X_train_std)

# Calculate the MAE for the Stacking Regressor
mae_stacked = mean_absolute_error(y_train, stacked_train_predictions)
print(f'MAE of Stacking Regressor: {mae_stacked:.6f}')

MAE of Stacking Regressor: 0.173959


In [126]:
from xgboost import XGBRegressor

# Define los modelos base
base_learners = [
    ('rf', RandomForestRegressor(max_depth=30,
                                 min_samples_leaf=1,
                                 min_samples_split=2,
                                 n_estimators=100,
                                 bootstrap=True,
                                 random_state=42)),
    ('gboost', GradientBoostingRegressor(learning_rate=0.1,
                                         max_depth=7,
                                         max_features='sqrt',
                                         min_samples_leaf=3,
                                         min_samples_split=2,
                                         n_estimators=100,
                                         subsample=0.9,
                                         random_state=42)),
    ('xgb', XGBRegressor(objective='reg:squarederror',
                         subsample=0.8,
                         min_child_weight=4,
                         max_depth=9,
                         learning_rate=0.1,
                         gamma=0,
                         colsample_bytree=0.8,
                         random_state=42))
]

# Inicializa el modelo de Stacking
stack_reg = StackingRegressor(
    estimators=base_learners,
    final_estimator=LinearRegression(),
    cv=5  # Utiliza validación cruzada con 5 folds para entrenar los modelos base
)


# Train the model
stack_reg.fit(X_train_std, y_train)

# Predict on the training set
stacked_train_predictions = stack_reg.predict(X_train_std)

# Calculate the MAE for the Stacking Regressor
mae_stacked = mean_absolute_error(y_train, stacked_train_predictions)
print(f'MAE of Stacking Regressor: {mae_stacked:.6f}')

MAE of Stacking Regressor: 0.175173


In [130]:
from xgboost import XGBRegressor

# Define los modelos base
base_learners = [
    ('rf', RandomForestRegressor(max_depth=30,
                                 min_samples_leaf=1,
                                 min_samples_split=2,
                                 n_estimators=100,
                                 bootstrap=True,
                                 random_state=42)),
    ('gboost', GradientBoostingRegressor(learning_rate=0.1,
                                         max_depth=7,
                                         max_features='sqrt',
                                         min_samples_leaf=3,
                                         min_samples_split=2,
                                         n_estimators=100,
                                         subsample=0.9,
                                         random_state=42)),
    ('xgb', XGBRegressor(objective='reg:squarederror',
                         subsample=0.8,
                         min_child_weight=4,
                         max_depth=9,
                         learning_rate=0.1,
                         gamma=0,
                         colsample_bytree=0.8,
                         random_state=42))
]

# Inicializa el modelo de Stacking
stack_reg = StackingRegressor(
    estimators=base_learners,
    final_estimator=LinearRegression(),
    cv=5  # Utiliza validación cruzada con 5 folds para entrenar los modelos base
)


# Train the model
stack_reg.fit(X_test_std, y_test)

# Predict on the training set
stacked_train_predictions = stack_reg.predict(X_test_std)

# Calculate the MAE for the Stacking Regressor
mae_stacked = mean_absolute_error(y_test, stacked_train_predictions)
print(f'MAE of Stacking Regressor: {mae_stacked:.6f}')

MAE of Stacking Regressor: 0.186861


## Conclusion

In [128]:
# Inicializar un DataFrame para registrar los resultados
results_df = pd.DataFrame(columns=['Model', 'MAE'])


# Función para añadir resultados al DataFrame
def add_result(model_name, mae_value):
    global results_df
    new_row = pd.DataFrame({'Model': [model_name], 'MAE': [mae_value]})
    results_df = pd.concat([results_df, new_row], ignore_index=True)


# Añadir los resultados (solo como ejemplo, reemplaza las cifras ficticias con tus valores reales)
add_result("Linear Regression", mae_lr)  # Sustituye 0.1234 con tu valor real
add_result("Ridge Regression", mae_ridge)  # Sustituye 0.1234 con tu valor real
add_result("Lasso Regression", mae_lasso)  # Sustituye 0.2345 con tu valor real
add_result("Decision Tree", mae_dt)  # Sustituye 0.2345 con tu valor real
add_result("Random Forest", mae_rf)  # Sustituye 0.2345 con tu valor real
add_result("SVR", mae_svr)  # Sustituye 0.2345 con tu valor real
add_result("AdaBoost", mae_adaboost)  # Sustituye 0.2345 con tu valor real
add_result("XGBoost", mae_optimized_xgb)  # Sustituye 0.2345 con tu valor real
add_result("Gradient Boost", mae_gb_std)  # Sustituye 0.2345 con tu valor real
add_result("Stacking", mae_stacked)  # Sustituye 0.2345 con tu valor real

# Muestra el DataFrame
sorted_results_df = results_df.sort_values(by="MAE", ascending=True)
print(sorted_results_df)

               Model       MAE
7            XGBoost  0.151644
9           Stacking  0.175173
4      Random Forest  0.187084
8     Gradient Boost  0.262176
5                SVR  0.567411
6           AdaBoost  0.597148
1   Ridge Regression  0.621813
0  Linear Regression  0.621816
3      Decision Tree  0.622449
2   Lasso Regression  0.650031
